# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [17]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API

In [18]:
challenge_id = 'myUniqueIdentifier1911283' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
query_results = query(challenge_id, queries)

# print(query_results)

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [19]:
best_query_number = np.argmax(query_results)
best_query = queries[best_query_number]
best_query_result = query(challenge_id, best_query, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")


Reconstruction attack achieves fraction 0.6 correct values


In [20]:
import numpy as np
from scipy.optimize import linprog
challenge_id = 'myUniqueIdentifier1911283' # identifier for hidden dataset
c=np.hstack((np.zeros(n),np.ones(num_queries)))
bounds=[(-1,1) for _ in range(n)]+[(0,None) for _ in range(num_queries)] #the range for data is -1 to 1, the range for the slack variables is 0 to infinity
A_ub=[]
b_ub=[]
# we are trying to minimize the sum of the slack variables, which is the sum of the last num_queries variables in c, AI used for getting inspiration
for i in range(num_queries):
    # First inequality: ⟨q_i, x⟩ - e_i ≤ y_i
    A_row_1 = np.hstack((queries[i], np.zeros(num_queries)))
    A_row_1[n + i] = -1 #-1 because we want to move the slack variable to the other side
    A_ub.append(A_row_1)
    # Second inequality: -⟨q_i, x⟩ - e_i ≤ -y_i
    A_row_2 = np.hstack((-queries[i], np.zeros(num_queries)))
    A_row_2[n + i] = -1
    A_ub.append(A_row_2)
    b_ub.append(query_results[i])
    b_ub.append(-query_results[i])
A_ub=np.array(A_ub)

result3=linprog(c,A_ub=A_ub,b_ub=b_ub,bounds=bounds,method='highs')
DATA=result3.x[:n]
DATA_reconstructed=np.sign(DATA)

In [21]:
my_best_query_result = query(challenge_id, DATA_reconstructed, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + my_best_query_result / n) / 2} correct values")



Reconstruction attack achieves fraction 0.85 correct values
